# Grado en Robótica - Visión Artificial
## Práctica 5: Reconocedor de texturas y su evaluación
### Autor: Adrián Losada Álvarez

## Importamos las librerías necesarias

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import cv2
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
import os
import mahotas
import pandas as pd
import Models_Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

## Nota: Estas listas se utilizarán al final del documento para calcular las métricas

In [ ]:
Haralick_BL_predictions = []
Haralick_BL_labels = []
Haralick_Mahalanobis_predictions = []
Haralick_Mahalanobis_labels = []
LBP_BL_predictions = []
LBP_BL_labels = []
LBP_Mahalanobis_predictions = []
LBP_Mahalanobis_labels = []

# Implementación de los dos algoritmos que reconocerán las texturas de las imágenes aportadas

## Primer descriptor: **Haralick (GLCM)**
Este descriptor funciona de la siguiente manera:
1. **Construcción de la matriz de co-ocurrencia**: Representa la frecuencia con la que aparecen pares de valores de intensidad de gris a una distnacia y en una dirección dadas.

2. **Normalización de la matriz**: La matriz se normaliza diviendo cada valor por el número total de pares de píxeles en la imagen, lo que da como resultado la probabilidad de que ocurra cada par de intensidades de gris.

3. **Cálculo de características**: A partir de la matriz normalizada, se calculan varias características que describne la textura de la región de interés, como la enegía, la homogeneidad, el contraste, etc...

4. **Análisis de características**: Estas características proporcionan información sobre la distribución de los niveles de girs en la imagne y se pueden utilizar para clasificar y comparar diferentes texturas.

### Definimos los paths y los diccionarios para codificar y decodificar las clases de texturas

In [ ]:
# Definir los paths al directorio train y test
train_path = './dataset/train/'
test_path = './dataset/test/'

# Diccionario para mapear clases de textura a números
texture_mapping = {'canvas1': 0.0, 'cushion1': 1.0, 'linseeds1': 2.0, 'sand1': 3.0, 'seat2': 4.0, 'stone1': 5.0}

# Diccionario inverso para mapear los números a las etiquetas de texto originales
inverse_texture_mapping = {0.0: 'canvas1', 1.0: 'cushion1', 2.0: 'linseeds1', 3.0: 'sand1', 4.0: 'seat2', 5.0: 'stone1'}

### Obtenemos las características de textura de las imágenes y las almacenamos en un archivo CSV

In [ ]:
print("[INFO] Obteniendo las características de textura...")
# Inicializamos la matriz de datos y la lista de etiquetas
data = []
labels = []

# Recorremos las imágenes de entrenamiento
for folderName in os.listdir(train_path):
    for imageName in os.listdir(train_path+folderName):
        # Definir el path a la imagen actual
        image_path = train_path+folderName+'/'+imageName

        # Cargamos la imagen, la convertimos a gris y extraemos el nombre de la textura a partir del nombre del fichero
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        texture = image_path[image_path.rfind("/") + 1:].split("_")[0].split("-")[0]

        # Mapeamos la textura a su número asociado
        label = texture_mapping.get(texture, -1)  # -1 si la textura no está mapeada

        # Si la textura está mapeada, calculamos las características de Haralick
        if label != -1:
            # Obtenemos las características de Haralick de la imagen
            features = mahotas.features.haralick(gray).mean(axis=0)

            # Actualizamos los datos y las etiquetas
            data.append(features)
            labels.append(label)

# Creamos un DataFrame para los datos y las etiquetas
dataset = pd.DataFrame(data=np.array(data), columns=[f'feature_{i}' for i in range(len(data[0]))])
dataset['texture'] = labels

# Guardamos el DataFrame
dataset.to_csv("./features/Haralick_features.csv", index=False)

### Normalizamos el archivo CSV de las características

In [ ]:
# Eliminamos las columnas que siempre tienen valor '0.0' en el archivo CSV
df = pd.read_csv('./features/Haralick_features.csv')

# Encuentra las columnas
columns_to_drop = []
for column in df.columns:
    if (df[column] == 0.0).all():
        columns_to_drop.append(column)

# Elimina las columnas encontradas
df = df.drop(columns=columns_to_drop)

# Guarda el DataFrame resultante
df.to_csv('./features/Haralick_features.csv', index=False)

# Normalizamos el CSV
Models_Metrics.normalizaCSV('./features/Haralick_features.csv', './features/Haralick_features_normalized.csv')

### Entrenamos el modelo con el **clasificador Bayesiano lineal**

In [ ]:
# Cargamos el CSV con las características
rawData = Models_Metrics.load_data('./features/Haralick_features_normalized.csv')
data = Models_Metrics.Data(rawData, bias=False)

print("[INFO] Entrenando el modelo...")
# Entrenamos el clasificador
HaralickByLinearModel = Models_Metrics.ByLinear(data)
HaralickByLinearModel.fix()

### Clasificamos las imágenes de prueba

In [ ]:
print("[INFO] Clasificando...")
# Contar el número de clases y el número de imágenes por clase
num_classes = len(os.listdir(test_path))
num_images_per_class = len(os.listdir(os.path.join(test_path, os.listdir(test_path)[0])))

# Obtenemos el valor máximo del CSV para normalizar posteriormente las características de las imágenes test
valor_maximo = Models_Metrics.obtener_valor_maximo('./features/Haralick_features.csv')

# Crear el subplot
fig, axs = plt.subplots(num_classes, num_images_per_class, figsize=(20, 15))

# Recorrer las imágenes de prueba y mostrarlas en el subplot
for i, folderName in enumerate(sorted(os.listdir(test_path))):
    class_images = []
    for j, imageName in enumerate(sorted(os.listdir(os.path.join(test_path, folderName)))):
        # Definir el path a la imagen actual
        image_path = os.path.join(test_path, folderName, imageName)

        # Cargamos la imagen, la convertimos a gris y extraemos el descriptor de Haralick de cada imagen de prueba
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = mahotas.features.haralick(gray).mean(axis=0)

        # Elimina las mismas características que se eliminaron en el archivo CSV de la lista de features
        new_features = []
        for idx, feature in enumerate(features):
            if ('feature_'+str(idx)) not in columns_to_drop:
                new_features.append(feature)

        # Normalizamos las características
        norm_features = np.array(new_features)/valor_maximo

        # Clasificamos la imagen de prueba
        pred = HaralickByLinearModel.predict(norm_features.reshape(1, -1))
        Haralick_BL_predictions.append(pred[0, 0])

        # Visualización de los resultados
        pred_text = "Predicted: " + inverse_texture_mapping[pred[0, 0]]  # Utilizando el diccionario inverso para pasar de número a texto de clase
        text_size = cv2.getTextSize(pred_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0] # Obtener el tamaño del texto
        background_coords = ((40, 50 - text_size[1]), (40 + text_size[0], 50))  # Calcular la posición del rectángulo de fondo
        cv2.rectangle(image, background_coords[0], background_coords[1], (0, 0, 0), -1) # Dibujar el rectángulo de fondo
        cv2.putText(image, pred_text, (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)  # Agregar el texto con la predicción

        # Añadir la imagen clasificada a la lista de imágenes de su clase
        class_images.append(image)

    # Mostrar las imágenes de la clase actual en una fila del subplot
    for j in range(num_images_per_class):
        axs[i, j].imshow(cv2.cvtColor(class_images[j], cv2.COLOR_BGR2RGB))
        axs[i, j].set_title("Texture: " + folderName)
        Haralick_BL_labels.append(texture_mapping[folderName])
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

### Entrenamos el modelo con la **métrica de Mahalanobis**

In [ ]:
# Cargamos el CSV con las características
rawData = Models_Metrics.load_data('./features/Haralick_features_normalized.csv')
data = Models_Metrics.Data(rawData, bias=False)

print("[INFO] Entrenando el modelo...")
# Entrenamos el clasificador
HaralickMahalanobisModel = Models_Metrics.MahalanobisMetric(data)
HaralickMahalanobisModel.fix()

### Clasificamos las imágenes de prueba

In [ ]:
print("[INFO] Clasificando...")
# Contar el número de clases y el número de imágenes por clase
num_classes = len(os.listdir(test_path))
num_images_per_class = len(os.listdir(os.path.join(test_path, os.listdir(test_path)[0])))

# Obtenemos el valor máximo del CSV para normalizar posteriormente las características de las imágenes test
valor_maximo = Models_Metrics.obtener_valor_maximo('./features/Haralick_features.csv')

# Crear el subplot
fig, axs = plt.subplots(num_classes, num_images_per_class, figsize=(20, 15))

# Recorrer las imágenes de prueba y mostrarlas en el subplot
for i, folderName in enumerate(sorted(os.listdir(test_path))):
    class_images = []
    for j, imageName in enumerate(sorted(os.listdir(os.path.join(test_path, folderName)))):
        # Definir el path a la imagen actual
        image_path = os.path.join(test_path, folderName, imageName)

        # Cargamos la imagen, la convertimos a gris y extraemos el descriptor de Haralick de cada imagen de prueba
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = mahotas.features.haralick(gray).mean(axis=0)

        # Elimina las mismas características que se eliminaron en el archivo CSV de la lista de features
        new_features = []
        for idx, feature in enumerate(features):
            if ('feature_'+str(idx)) not in columns_to_drop:
                new_features.append(feature)

        # Normalizamos las características
        norm_features = np.array(new_features)/valor_maximo

        # Clasificamos la imagen de prueba
        pred = HaralickMahalanobisModel.predict(norm_features.reshape(1, -1))
        Haralick_Mahalanobis_predictions.append(pred[0])

        # Visualización de los resultados
        pred_text = "Predicted: " + inverse_texture_mapping[pred[0]]  # Utilizando el diccionario inverso para pasar de número a texto de clase
        text_size = cv2.getTextSize(pred_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0] # Obtener el tamaño del texto
        background_coords = ((40, 50 - text_size[1]), (40 + text_size[0], 50))  # Calcular la posición del rectángulo de fondo
        cv2.rectangle(image, background_coords[0], background_coords[1], (0, 0, 0), -1) # Dibujar el rectángulo de fondo
        cv2.putText(image, pred_text, (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)  # Agregar el texto con la predicción

        # Añadir la imagen clasificada a la lista de imágenes de su clase
        class_images.append(image)

    # Mostrar las imágenes de la clase actual en una fila del subplot
    for j in range(num_images_per_class):
        axs[i, j].imshow(cv2.cvtColor(class_images[j], cv2.COLOR_BGR2RGB))
        axs[i, j].set_title("Texture: " + folderName)
        Haralick_Mahalanobis_labels.append(texture_mapping[folderName])
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

### Segundo descriptor: **Linear Binary Patterns (LBP)**
Este descriptor funciona de la siguiente manera:
1. **Definición de vecindarios**: Se define un vecindario alrededor de cada píxel de la imagen. Este vecindario puede ser un círculo, un cuadrado o cualquier forma definida por el usuario. Comúnmente, se utiliza un vecindario circular. Además, el tamaño de este puede variar.

2. **Cálculo de los patrones binarios**: Para cada píxel de la imagen, se compara su valor de intensidad con los valores de intensidad de sus vecinos en el vecindario definido. Se asigna un bit (1 o 0) a cada vecino según si su intensidad es mayor o menor que la del píxel central respectivamente. Estos bits se concatenan para formar un número binario.

3. **Histograma de patrones binarios**: Se construye un histograma contando la frecuencia de cada patrón binario en la imagen. Este histograma representa la distribución de los patrones locales en la imagen.

4. **Características LBP**: A partir del histograma de patrones binarios, se pueden calcular diversas características para describir la textura de la imagen, como la media, la varianza, etc. También es común utilizar directamente el histograma como un descriptor de la textura.

En este caso utilizaremos un radio de valor 1 y 8 puntos

#### Definimos los paths y los diccionarios para codificar y decodificar las clases de texturas

In [ ]:
# Definir los paths al directorio train y test
train_path = './dataset/train/'
test_path = './dataset/test/'

# Diccionario para mapear clases de textura a números
texture_mapping = {'canvas1': 0, 'cushion1': 1, 'linseeds1': 2, 'sand1': 3, 'seat2': 4, 'stone1': 5}

# Diccionario inverso para mapear los números a las etiquetas de texto originales
inverse_texture_mapping = {0: 'canvas1', 1: 'cushion1', 2: 'linseeds1', 3: 'sand1', 4: 'seat2', 5: 'stone1'}

#### Obtenemos las características de textura de las imágenes y las almacenamos en un archivo CSV

In [ ]:
print("[INFO] Obteniendo las características de textura...")
# Inicializamos la matriz de datos y la lista de etiquetas
data = []
labels = []

# Recorremos las imágenes de entrenamiento
for folderName in os.listdir(train_path):
    for imageName in os.listdir(train_path+folderName):
        # Definir el path a la imagen actual
        image_path = train_path+folderName+'/'+imageName

        # Cargamos la imagen, la convertimos a gris y extraemos el nombre de la textura a partir del nombre del fichero
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        texture = image_path[image_path.rfind("/") + 1:].split("_")[0].split("-")[0]

        # Mapeamos la textura a su número asociado
        label = texture_mapping.get(texture, -1)  # -1 si la textura no está mapeada

        # Si la textura está mapeada, calculamos las características de Haralick
        if label != -1:
            # Extraemos las características de LBP con un radio de 1 y 8 puntos
            features = mahotas.features.lbp(gray, 1, 8)

            # Actualizamos los datos y las etiquetas
            data.append(features)
            labels.append(label)

# Creamos un DataFrame para los datos y las etiquetas
dataset = pd.DataFrame(data=np.array(data), columns=[f'feature_{i}' for i in range(len(data[0]))])
dataset['texture'] = labels

# Guardamos el DataFrame en un archivo CSV
dataset.to_csv("./features/LBP_features.csv", index=False)

#### Normalizamos el archivo CSV de las características

In [ ]:
# Eliminamos las columnas que siempre tienen valor '0.0' en el archivo CSV
df = pd.read_csv('./features/LBP_features.csv')

# Encuentra las columnas
columns_to_drop = []
for column in df.columns:
    if (df[column] == 0.0).all():
        columns_to_drop.append(column)

# Elimina las columnas encontradas
df = df.drop(columns=columns_to_drop)

# Guarda el DataFrame resultante
df.to_csv('./features/LBP_features.csv', index=False)

# Normalizamos el CSV
Models_Metrics.normalizaCSV('./features/LBP_features.csv', './features/LBP_features_normalized.csv')

#### Entrenamos el modelo con el **clasificador Bayesiano lineal**

In [ ]:
# Cargamos el CSV con las características
rawData = Models_Metrics.load_data('./features/LBP_features_normalized.csv')
data = Models_Metrics.Data(rawData, bias=False)

print("[INFO] Entrenando el modelo...")
# Entrenamos el clasificador
LBPByLinearModel = Models_Metrics.ByLinear(data)
LBPByLinearModel.fix()

#### Clasificamos las imágenes de prueba

In [ ]:
print("[INFO] Clasificando...")
# Contar el número de clases y el número de imágenes por clase
num_classes = len(os.listdir(test_path))
num_images_per_class = len(os.listdir(os.path.join(test_path, os.listdir(test_path)[0])))

# Obtenemos el valor máximo del CSV para normalizar posteriormente las características de las imágenes test
valor_maximo = Models_Metrics.obtener_valor_maximo('./features/LBP_features.csv')

# Crear el subplot
fig, axs = plt.subplots(num_classes, num_images_per_class, figsize=(20, 15))

# Recorrer las imágenes de prueba y mostrarlas en el subplot
for i, folderName in enumerate(sorted(os.listdir(test_path))):
    class_images = []
    for j, imageName in enumerate(sorted(os.listdir(os.path.join(test_path, folderName)))):
        # Definir el path a la imagen actual
        image_path = os.path.join(test_path, folderName, imageName)

        # Cargamos la imagen, la convertimos a gris y extraemos las características con el descriptor LBP de cada imagen de prueba
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = mahotas.features.lbp(gray, 1, 8)

        # Elimina las mismas características que se eliminaron en el archivo CSV de la lista de features
        new_features = []
        for idx, feature in enumerate(features):
            if ('feature_'+str(idx)) not in columns_to_drop:
                new_features.append(feature)

        # Normalizamos las características
        norm_features = np.array(new_features)/valor_maximo

        # Clasificamos la imagen de prueba
        pred = LBPByLinearModel.predict(norm_features.reshape(1, -1))
        LBP_BL_predictions.append(pred[0, 0])

        # Visualización de los resultados
        pred_text = "Predicted: " + inverse_texture_mapping[pred[0, 0]]  # Utilizando el diccionario inverso para pasar de número a texto de clase
        text_size = cv2.getTextSize(pred_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0] # Obtener el tamaño del texto
        background_coords = ((40, 50 - text_size[1]), (40 + text_size[0], 50))  # Calcular la posición del rectángulo de fondo
        cv2.rectangle(image, background_coords[0], background_coords[1], (0, 0, 0), -1) # Dibujar el rectángulo de fondo
        cv2.putText(image, pred_text, (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)  # Agregar el texto con la predicción

        # Añadir la imagen clasificada a la lista de imágenes de su clase
        class_images.append(image)

    # Mostrar las imágenes de la clase actual en una fila del subplot
    for j in range(num_images_per_class):
        axs[i, j].imshow(cv2.cvtColor(class_images[j], cv2.COLOR_BGR2RGB))
        axs[i, j].set_title("Correct class: " + folderName)
        LBP_BL_labels.append(texture_mapping[folderName])
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

### Entrenamos el modelo con la **métrica de Mahalanobis**

In [ ]:
# Cargamos el CSV con las características
rawData = Models_Metrics.load_data('./features/LBP_features_normalized.csv')
data = Models_Metrics.Data(rawData, bias=False)

print("[INFO] Entrenando el modelo...")
# Entrenamos el clasificador
LBPMahalanobisModel = Models_Metrics.MahalanobisMetric(data, alpha=1e-6)
LBPMahalanobisModel.fix()

#### Clasificamos las imágenes de prueba

In [ ]:
print("[INFO] Clasificando...")
# Contar el número de clases y el número de imágenes por clase
num_classes = len(os.listdir(test_path))
num_images_per_class = len(os.listdir(os.path.join(test_path, os.listdir(test_path)[0])))

# Obtenemos el valor máximo del CSV para normalizar posteriormente las características de las imágenes test
valor_maximo = Models_Metrics.obtener_valor_maximo('./features/LBP_features.csv')

# Crear el subplot
fig, axs = plt.subplots(num_classes, num_images_per_class, figsize=(20, 15))

# Recorrer las imágenes de prueba y mostrarlas en el subplot
for i, folderName in enumerate(sorted(os.listdir(test_path))):
    class_images = []
    for j, imageName in enumerate(sorted(os.listdir(os.path.join(test_path, folderName)))):
        # Definir el path a la imagen actual
        image_path = os.path.join(test_path, folderName, imageName)

        # Cargamos la imagen, la convertimos a gris y extraemos las características con el descriptor LBP de cada imagen de prueba
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = mahotas.features.lbp(gray, 1, 8)

        # Elimina las mismas características que se eliminaron en el archivo CSV de la lista de features
        new_features = []
        for idx, feature in enumerate(features):
            if ('feature_'+str(idx)) not in columns_to_drop:
                new_features.append(feature)

        # Normalizamos las características
        norm_features = np.array(new_features)/valor_maximo

        # Clasificamos la imagen de prueba
        pred = LBPMahalanobisModel.predict(norm_features.reshape(1, -1))
        LBP_Mahalanobis_predictions.append(pred[0])

        # Visualización de los resultados
        pred_text = "Predicted: " + inverse_texture_mapping[pred[0]]  # Utilizando el diccionario inverso para pasar de número a texto de clase
        text_size = cv2.getTextSize(pred_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0] # Obtener el tamaño del texto
        background_coords = ((40, 50 - text_size[1]), (40 + text_size[0], 50))  # Calcular la posición del rectángulo de fondo
        cv2.rectangle(image, background_coords[0], background_coords[1], (0, 0, 0), -1) # Dibujar el rectángulo de fondo
        cv2.putText(image, pred_text, (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)  # Agregar el texto con la predicción

        # Añadir la imagen clasificada a la lista de imágenes de su clase
        class_images.append(image)

    # Mostrar las imágenes de la clase actual en una fila del subplot
    for j in range(num_images_per_class):
        axs[i, j].imshow(cv2.cvtColor(class_images[j], cv2.COLOR_BGR2RGB))
        axs[i, j].set_title("Texture: " + folderName)
        LBP_Mahalanobis_labels.append(texture_mapping[folderName])
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

## Medidas de rendimiento

### **Obtención de métricas:**

#### Modelo: **Haralick**
#### Clasificador: **Bayesiano Lineal**

In [ ]:
Haralick_BL_accuracy = accuracy_score(np.array(Haralick_BL_labels), np.array(Haralick_BL_predictions))
Haralick_BL_precision = precision_score(np.array(Haralick_BL_labels), np.array(Haralick_BL_predictions), average='micro')
Haralick_BL_recall = recall_score(np.array(Haralick_BL_labels), np.array(Haralick_BL_predictions), average='micro')
Haralick_BL_f1_score = f1_score(np.array(Haralick_BL_labels), np.array(Haralick_BL_predictions), average='micro')
Haralick_BL_confusion_matrix = confusion_matrix(np.array(Haralick_BL_labels), np.array(Haralick_BL_predictions))

##### Modelo: **Haralick**
##### Clasificador: **Mahalanobis**

In [ ]:
Haralick_Mahalanobis_accuracy = accuracy_score(np.array(Haralick_Mahalanobis_labels), np.array(Haralick_Mahalanobis_predictions))
Haralick_Mahalanobis_precision = precision_score(np.array(Haralick_Mahalanobis_labels), np.array(Haralick_Mahalanobis_predictions), average='micro')
Haralick_Mahalanobis_recall = recall_score(np.array(Haralick_Mahalanobis_labels), np.array(Haralick_Mahalanobis_predictions), average='micro')
Haralick_Mahalanobis_f1_score = f1_score(np.array(Haralick_Mahalanobis_labels), np.array(Haralick_Mahalanobis_predictions), average='micro')
Haralick_Mahalanobis_confusion_matrix = confusion_matrix(np.array(Haralick_Mahalanobis_labels), np.array(Haralick_Mahalanobis_predictions))

##### Modelo: **LBP**
##### Clasificador: **Bayesiano Lineal**

In [ ]:
LBP_BL_accuracy = accuracy_score(np.array(LBP_BL_labels), np.array(LBP_BL_predictions))
LBP_BL_precision = precision_score(np.array(LBP_BL_labels), np.array(LBP_BL_predictions), average='micro')
LBP_BL_recall = recall_score(np.array(LBP_BL_labels), np.array(LBP_BL_predictions), average='micro')
LBP_BL_f1_score = f1_score(np.array(LBP_BL_labels), np.array(LBP_BL_predictions), average='micro')
LBP_BL_confusion_matrix = confusion_matrix(np.array(LBP_BL_labels), np.array(LBP_BL_predictions))

##### Modelo: **LBP**
##### Clasificador: **Mahalanobis**

In [ ]:
LBP_Mahalanobis_accuracy = accuracy_score(np.array(LBP_Mahalanobis_labels), np.array(LBP_Mahalanobis_predictions))
LBP_Mahalanobis_precision = precision_score(np.array(LBP_Mahalanobis_labels), np.array(LBP_Mahalanobis_predictions), average='micro')
LBP_Mahalanobis_recall = recall_score(np.array(LBP_Mahalanobis_labels), np.array(LBP_Mahalanobis_predictions), average='micro')
LBP_Mahalanobis_f1_score = f1_score(np.array(LBP_Mahalanobis_labels), np.array(LBP_Mahalanobis_predictions), average='micro')
LBP_Mahalanobis_confusion_matrix = confusion_matrix(np.array(LBP_Mahalanobis_labels), np.array(LBP_Mahalanobis_predictions))

### Mostramos los resultados

In [ ]:
# Listas
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
models = ['Haralick+BayeLinear(13 características)', 'Haralick+Mahalanobis(13 características)', 'LBP+BayeLinear(35 características)', 'LBP+Mahalanobis(35 características)']
score_lists = [
    [Haralick_BL_accuracy, Haralick_Mahalanobis_accuracy, LBP_BL_accuracy, LBP_Mahalanobis_accuracy],
    [Haralick_BL_precision, Haralick_Mahalanobis_precision, LBP_BL_precision, LBP_Mahalanobis_precision],
    [Haralick_BL_recall, Haralick_Mahalanobis_recall, LBP_BL_recall, LBP_Mahalanobis_recall],
    [Haralick_BL_f1_score, Haralick_Mahalanobis_f1_score, LBP_BL_f1_score, LBP_Mahalanobis_f1_score]
]

# Dataframe con las métricas
data = {'Modelo': models}
for metric, score_list in zip(metrics, score_lists):
    sorted_scores = sorted(score_list, reverse=True)
    data[metric] = score_list
    data[f'{metric.rjust(2)} Rank'] = [sorted_scores.index(score) + 1 for score in score_list]

df = pd.DataFrame(data)

# Mostramos el DataFrame
df

#### Matrices de confusión:

In [ ]:
# Definimos la lista con las matrices de confusión y los nombres correspondientes
list_CM = [Haralick_BL_confusion_matrix, Haralick_Mahalanobis_confusion_matrix, LBP_BL_confusion_matrix, LBP_Mahalanobis_confusion_matrix]
names = ['Haralick - Bayesiano Lineal', 'Haralick - Métrica Mahalanobis', 'Linear Binary Patterns - Bayesiano Lineal', 'Linear Binary Patterns - Métrica Mahalanobis']

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
for i in range(2):
    for j in range(2):
        # Crea el heatmap utilizando seaborn
        sns.heatmap(list_CM[i*2 + j], annot=True, cmap='Blues', fmt='g', 
                    xticklabels=['canvas1', 'cushion1', 'linseeds1', 'sand1', 'seat2', 'stone1'], 
                    yticklabels=['canvas1', 'cushion1', 'linseeds1', 'sand1', 'seat2', 'stone1'], ax=axes[i, j])
        
        # Etiquetas y título
        axes[i, j].set_xlabel('Predicción')
        axes[i, j].set_ylabel('Etiqueta Verdadera')
        axes[i, j].set_title(names[i*2 + j])

# Mostramos el resultado
plt.tight_layout()
plt.show()


#### Análisis de los resultados
Observando los resultados podemos comprobar que, para la base de datos aportada, los dos descriptores con el clasificador Bayesiano lineal son capaces de identificar sin problema las distintas texturas. Utilizando la métrica de Mahalanobis como clasificador vemos que el resultado sigue siendo muy bueno, sin embargo, en el caso del descriptor de Halarick podemos comprobar que si ha tenido algún fallo a la hora de predecir.

Estas métricas tan elevadas se pueden deber a que la base de datos tiene una dimensión bastante pequeña, además de que todas las imágenes de texturas estaban a la misma escala y misma orientación. Estos factores facilitan el reconocimiento y diferenciación de las distintas texturas.